In [ ]:
# from scipy.cluster import  hierarchy

# #Vectorizing
# X = CountVectorizer().fit_transform(docs)
# X = TfidfTransformer().fit_transform(X)
# #Clustering
# X = X.todense()
# threshold = 0.1
# Z = hierarchy.linkage(X,"average", metric="cosine")
# C = hierarchy.fcluster(Z, threshold, criterion="distance")

In [6]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from sklearn.cluster import KMeans
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm
import re
import nltk
import pickle

In [7]:
bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'embeddings/rubert_cased_L-12_H-768_A-12_pt'

m = build_model(bert_config)

dramas = [f for f in listdir('drama/') if isfile(join('drama/', f))]
dramas_texts = []


for drama in dramas:
    with open(join('drama/', drama), 'rb') as f:
        dramas_texts.append(pickle.load(f))

dramas_texts[0][0][:5]

[('Григорий', 'Ступай отвори дверь! звонят!'),
 ('Петр',
  'Да ты что сидишь? На ногах у тебя пузыри, что ли? встать не можешь?'),
 ('Иван', 'Ну, уж я пойду, так и быть, отворю! Это Андрюшка!'),
 ('Григорий', 'А, московская ворона! Откуда тебя принесло?'),
 ('Чужой слуга',
  'Ах ты, чухонский сын! Побегал бы ты с мое. Вот к цветочнице велела снесть, что на Петербургской. Небось четвертака на извозчика не\n            даст. Да и к вашему тож. Что, спит?')]

In [8]:
only_sents = []

for drama in tqdm(dramas_texts):
    for part in drama:
        only_sents = only_sents + [sent for _, sent in part]
        
only_sents[:5]

['Ступай отвори дверь! звонят!',
 'Да ты что сидишь? На ногах у тебя пузыри, что ли? встать не можешь?',
 'Ну, уж я пойду, так и быть, отворю! Это Андрюшка!',
 'А, московская ворона! Откуда тебя принесло?',
 'Ах ты, чухонский сын! Побегал бы ты с мое. Вот к цветочнице велела снесть, что на Петербургской. Небось четвертака на извозчика не\n            даст. Да и к вашему тож. Что, спит?']

In [9]:
only_sents_splited = []

for text in tqdm(only_sents):
    only_sents_splited = only_sents_splited + nltk.sent_tokenize(re.sub('[\n]', ' ', text))
    
only_sents_splited[:5]

['Ступай отвори дверь!',
 'звонят!',
 'Да ты что сидишь?',
 'На ногах у тебя пузыри, что ли?',
 'встать не можешь?']

In [45]:
%time _, _, _, _, _, only_sents_vectors_mean_tmp, _ = m(only_sents_splited[0:20])

CPU times: user 1.49 s, sys: 24.1 ms, total: 1.52 s
Wall time: 261 ms


In [38]:
only_sents_vectors_mean_tmp[19]

array([ 3.40485930e-01,  2.89081763e-02, -1.55306403e-02,  5.00403419e-02,
        4.73519504e-01,  2.92812914e-01,  2.17403367e-01,  1.27317697e-01,
        3.17242503e-01,  1.19579263e-01,  4.13464725e-01, -4.89334255e-01,
        2.10110217e-01,  2.69542038e-02, -3.36438678e-02,  2.10082248e-01,
        8.20645690e-01, -4.00277227e-02, -2.57219136e-01,  2.25814342e-01,
        1.76613480e-01,  1.90805405e-01,  1.95128158e-01,  1.14873312e-01,
       -7.48332292e-02,  6.07909143e-01,  2.84835100e-01, -1.99620165e-02,
        1.11015938e-01, -1.43942945e-02,  1.10531397e-01,  9.85679448e-01,
       -1.55058369e-01,  1.63893551e-01, -1.97096020e-01, -1.40421242e-01,
       -1.51251301e-01,  2.62875050e-01, -4.67013687e-01, -1.43368756e-02,
       -2.07804088e-02,  3.40785652e-01,  2.80349031e-02,  2.18807831e-02,
       -4.67282124e-02,  1.28386885e-01,  3.30323100e-01,  6.96432054e-01,
       -4.81581211e-01,  1.52867451e-01,  4.31912839e-01,  9.09081176e-02,
       -6.72585666e-01, -

In [16]:
a = [11,2]

In [26]:
int(sents_count / 250)

1057

In [47]:
only_sents_vectors_mean = []
sents_count = len(only_sents_splited)

for i in tqdm(range(int(sents_count / 5))):
    _, _, _, _, _, only_sents_vectors_mean_tmp, _ = m([only_sents_splited[i * 5: (i+1)*5]])
    only_sents_vectors_mean.extend(only_sents_vectors_mean_tmp)
    
with open('ru_bert_fullsplited_vectors.pkl', 'wb') as f:
    pickle.dump(zip(only_sents_splited, only_sents_vectors_mean), f)
    
only_sents_vectors_mean[:1]

[array([ 3.39768492e-02, -2.38059551e-01,  1.60009131e-01, -6.02096468e-02,
        -7.46953338e-02,  1.02441072e-01, -1.62181631e-01,  4.90932912e-02,
         3.85603011e-01,  2.80015528e-01,  3.54283303e-01, -5.64004958e-01,
         1.53780922e-01, -7.63937980e-02,  5.16246669e-02, -8.63554999e-02,
         5.72201014e-01, -2.83349436e-02, -1.14053212e-01,  5.18320739e-01,
        -1.07604802e-01, -2.66776651e-01, -1.61179274e-01, -1.21598788e-01,
        -1.94413364e-01,  3.35835218e-01,  1.47443593e-01, -2.17397243e-01,
         5.26882350e-01, -1.36143327e-01,  1.58247337e-01,  2.61795759e-01,
        -5.91260642e-02,  2.90794790e-01, -5.05444817e-02, -1.41488975e-02,
         2.92516112e-01,  1.82200879e-01, -2.15983376e-01,  4.60547656e-01,
         3.65842842e-02,  1.79963652e-02,  9.92076620e-02, -1.13271512e-01,
         2.49229670e-02, -2.42265522e-01,  1.35866776e-01,  1.44849792e-01,
        -5.88797033e-01, -3.74012947e-01,  9.30704549e-02,  2.37185508e-01,
        -5.6

In [5]:
with open('ru_bert_fullsplited_vectors.pkl', 'rb') as f:
    sents_vectors = pickle.load(f)

EOFError: Ran out of input

In [49]:
sents_vectors = list(zip(only_sents_splited, only_sents_vectors_mean))
sents_vectors[:1]

[('Ступай отвори дверь!',
  array([ 3.39768492e-02, -2.38059551e-01,  1.60009131e-01, -6.02096468e-02,
         -7.46953338e-02,  1.02441072e-01, -1.62181631e-01,  4.90932912e-02,
          3.85603011e-01,  2.80015528e-01,  3.54283303e-01, -5.64004958e-01,
          1.53780922e-01, -7.63937980e-02,  5.16246669e-02, -8.63554999e-02,
          5.72201014e-01, -2.83349436e-02, -1.14053212e-01,  5.18320739e-01,
         -1.07604802e-01, -2.66776651e-01, -1.61179274e-01, -1.21598788e-01,
         -1.94413364e-01,  3.35835218e-01,  1.47443593e-01, -2.17397243e-01,
          5.26882350e-01, -1.36143327e-01,  1.58247337e-01,  2.61795759e-01,
         -5.91260642e-02,  2.90794790e-01, -5.05444817e-02, -1.41488975e-02,
          2.92516112e-01,  1.82200879e-01, -2.15983376e-01,  4.60547656e-01,
          3.65842842e-02,  1.79963652e-02,  9.92076620e-02, -1.13271512e-01,
          2.49229670e-02, -2.42265522e-01,  1.35866776e-01,  1.44849792e-01,
         -5.88797033e-01, -3.74012947e-01,  9.3070

In [74]:
# clust_model_kmeans = KMeans(n_clusters=20, random_state=42)
# clust_model_kmeans.fit(only_sents_vectors_mean)
clusters = clust_model_kmeans.predict(only_sents_vectors_mean)
clusters[:5]

array([16, 16,  2,  5,  5], dtype=int32)

In [76]:
sents_with_clusters = list(zip([sent for sent, _ in sents_vectors], clusters))

In [77]:
for curr_cluster in sorted(list(set(clusters))):
    print(curr_cluster)
    sents_in_this_cluster = [sent for sent, cluster in sents_with_clusters if cluster==curr_cluster]
    for sent in sents_in_this_cluster[:10]:
        print(sent)
    print('=========================\n=========================\n=========================\n\n\n')

0
А, московская ворона!
Небось четвертака на извозчика не             даст.
Ну, уж ты, брат, будь теперь штопальница.
Не сидеть же без дела.
Конечно, я и лакей,             да и женский портной вместе.
Нет, брат, у хорошего барина лакея не займут работой, на то есть мастеровой.
Да.
звонит.
Подождет.
Ну, развязывай мошну, ты, штопальница!



1
медведь?
Тс!
Молчите!..
часы бьют.
Семь часов.
(Цензор изменил: вместо славянского «живота своего» поставил               «жизни своей».
Однако ведь о «животе» говорят не только в молитвах, но даже тогда,               когда «кладут его на алтарь отечества».
Спасибо тебе за то, что ты такой внимательный.
Тише, тише, мой батюшка!..
Прощайте, князь; благодарствуйте за             мыло.



2
Да ты что сидишь?
Ну, уж я пойду, так и быть, отворю!
Это Андрюшка!
Побегал бы ты с мое.
Нет, еще не рычал из берлоги.
Вон у             графа Булкина — тридцать, брат, человек слуг одних; и уж там, брат, нельзя так: «Эй,             Петрушка, сходи-ка туды».
сту

In [81]:
def cluster_text(per_text, bert_model, cluster_model):
    for person, text in per_text:
        sents = {sent: -1 for sent in nltk.sent_tokenize(re.sub('[\n]', ' ', text))}
        for sent in sents:
            _, _, _, _, _, only_sents_vectors_mean_tmp, _ = m([sent])
            cluster = cluster_model.predict(only_sents_vectors_mean_tmp)
            sents[sent] = cluster[0]
        print('{}: {}'.format(person, sents))
#     return text

cluster_text(dramas_texts[54][2], m, clust_model_kmeans)

Любим: {'Княгиня... дядюшка, угодно ли вам кушать?': 3, 'Я завтрака хотел добиться своего, И что же?': 3, 'Не нашел в столовой никого.': 14, 'Ваш дом вдруг сделался ужасною пустыней.': 14}
Княгиня: {'Но мы условились...': 1}
Граф: {'Да, братец, мы с княгиней Хотели осмотреть окружные леса, И завтрак в два часа назначен...': 10}
Любим: {'В два часа!': 14, 'Да я не доживу.- Мигайте как хотите; Вам, право, хорошо: вы по-московски спите, До десяти часов; а встаньте-ка с зарей, Так есть захочется.': 11}
Княгиня: {'Но разве?..': 14}
Любим: {'Боже мой!': 11, 'Я в пять часов с ружьем шатался по болоту.': 14}
Княгиня: {'Вы правы: он не спит.': 14}
Любим: {'Сбираясь на охоту, Вчера садовнику велел нарочно я, Чтоб ровно в семь часов он разбудил меня.': 10, 'Что ж?': 14, 'Слышу, до свету он мне кричит в окошко: "Вставайте, вам уж спать осталося немножко".': 11, 'Таким вниманием я столько был взбешен, Что встал не мешкая... а видел славный сон!': 6}
Граф: {'Вы слышите?': 14}
Княгиня: {'А что ж вы в

In [ ]:
len(only_sents_vectors_mean[0])

In [ ]:
help(pickle.dump)